In [3]:
import pandas as pd
import numpy as np
NaN = np.nan

# Read data
# Download the July 2019 data for FHVHV at https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.pag
c = pd.read_csv('fhvhv_tripdata_2019-07.csv')


In [4]:
# Drop unnecessary columns and trips from/to unknown area
try:
    c = c.drop(columns=['dispatching_base_num','SR_Flag'])
except:
    pass
c = c[c.PULocationID < 264]
c = c[c.DOLocationID < 264]


In [5]:
# Load ID-Coord mapping
ids = pd.read_csv('taxi_zones_simpler.csv')

In [6]:
# Replace location IDs with coordinates
for row in range(len(c)):
    if (row + 1) % 100000 == 0:
        print(row + 1, end=' ')
        #break
    try:
        c.at[row,'hvfhs_license_num'] = c.at[row,'hvfhs_license_num'][-1:]
        c.at[row,'PULocationID'] = ids.at[c.at[row,'PULocationID']-1,'K']
        c.at[row,'DOLocationID'] = ids.at[c.at[row,'DOLocationID']-1,'K']
    except KeyError:
        pass

c.head(5)

100000 200000 300000 400000 500000 600000 700000 800000 900000 1000000 1100000 1200000 1300000 1400000 1500000 1600000 1700000 1800000 1900000 2000000 2100000 2200000 2300000 2400000 2500000 2600000 2700000 2800000 2900000 3000000 3100000 3200000 3300000 3400000 3500000 3600000 3700000 3800000 3900000 4000000 4100000 4200000 4300000 4400000 4500000 4600000 4700000 4800000 4900000 5000000 5100000 5200000 5300000 5400000 5500000 5600000 5700000 5800000 5900000 6000000 6100000 6200000 6300000 6400000 6500000 6600000 6700000 6800000 6900000 7000000 7100000 7200000 7300000 7400000 7500000 7600000 7700000 7800000 7900000 8000000 8100000 8200000 8300000 8400000 8500000 8600000 8700000 8800000 8900000 9000000 9100000 9200000 9300000 9400000 9500000 9600000 9700000 9800000 9900000 10000000 10100000 10200000 10300000 10400000 10500000 10600000 10700000 10800000 10900000 11000000 11100000 11200000 11300000 11400000 11500000 11600000 11700000 11800000 11900000 12000000 12100000 12200000 12300000 1

,hvfhs_license_num,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID
0,3,2019-07-01 00:12:33,2019-07-01 00:25:00,7,7
1,3,2019-07-01 00:41:26,2019-07-01 00:51:21,1,6
2,3,2019-07-01 00:18:50,2019-07-01 00:32:48,0,0
3,5,2019-07-01 00:05:12,2019-07-01 00:19:26,0,0
4,5,2019-07-01 00:29:01,2019-07-01 00:45:50,0,4


In [7]:
c.to_csv("trips.csv")


class BFLView(FlaskView): #
    def __init__(self):
        # Generate a globally unique address for this node
        self.node_identifier = str(uuid4()).replace('-', '')

        # Instantiate the Blockchain
        self.blockchain = Blockchain(url="0.0.0.0:5001/bfl/")
        
    def index(self):
        return "<p>Test Page</p>"

    @route('/mine', methods=['POST'])
    def mine(self):
        # MINE should be POST???
        block_ = toBlock(request.get_json())

        # check PoA
        if self.blockchain.proofOfAuthority(block_['sender']) == 0:
            return "Error: unknown node.", 400

        # check PoQ
        quality = self.blockchain.proofOfQuality(block_)
        if quality == 0:
            return "Not mined: Substandard quality.", 400

        # Success: PoA, PoQ
        # Incentive
        self.blockchain.newTransaction("0", block_["sender"],'Incentive','payment', int(quality))

        # Forge the new Block by adding it to the chain
        previous_hash = str(hash(self.lastBlock()))
        proof = 0
        block = self.blockchain.newBlock(proof, previous_hash) # PROOF LATER_-----------------------------------------------------------------

        response = {
            'message': "New Block Forged",
            'index': block['index'],
            'transactions': block['transactions'],
            'proof': block['proof'],
            'previous_hash': block['previous_hash'],
        }
        # notify clients : notify()
        
        return jsonify(response), 200

    @route('/transactions/new', methods=['POST'])
    def newTransaction(self):
        values = request.get_json()

        # Create a new Transaction
        index = self.blockchain.newTransaction(values['sender'], values['recipient'], values['amount'])

        response = {'message': f'Transaction will be added to Block {index}'}
        return jsonify(response), 201

    @route('/chain', methods=['GET'])
    def fullChain(self):
        response = {
            'chain': self.blockchain.chain,
            'length': len(self.blockchain.chain),
        }
        return jsonify(response), 200

    @route('/nodes/register', methods=['POST'])
    def registerNodes(self):
        values = request.get_json()

        nodes = values.get('nodes')
        if nodes is None:
            return "Error: Please supply a valid list of nodes", 400

        for node in nodes:
            self.blockchain.authorizeClient(node)

        response = {
            'message': 'New nodes have been added',
            'total_nodes': list(self.blockchain.nodes),
        }
        return jsonify(response), 201


    @route('/nodes/resolve', methods=['GET'])
    def consensus(self):
        replaced = self.blockchain.resolveConflicts()

        if replaced:
            response = {
                'message': 'Our chain was replaced',
                'new_chain': self.blockchain.chain
            }
        else:
            response = {
                'message': 'Our chain is authoritative',
                'chain': self.blockchain.chain
            }

        return jsonify(response), 200

#app = Flask(__name__)
#x = BFLView.register(app)
#app.run('0.0.0.0',5001)